In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
#Mini Data Set
data = {
    "slope_angle": [45,20,55,30,15,40,60,25,50,35,10,65,28,48,38,52,18,42,58,33],
    "rainfall_mm": [5,5,30,10,2,0,35,8,20,12,0,40,9,18,14,22,3,6,28,11],
    "vibration": [0.03,0.01,0.14,0.03,0.005,0.01,0.16,0.02,0.1,0.05,0.0,0.18,0.025,0.09,0.06,0.11,0.007,0.035,0.13,0.045],
    "pore_pressure": [100,60,170,90,50,80,180,70,150,120,40,200,85,140,125,155,55,105,165,115],
    "temperature_c": [29,27,34,29,26,28,35,28,32,30,25,36,28,31,30,33,26,29,34,30]
}
df_small = pd.DataFrame(data)
df_small


,slope_angle,rainfall_mm,vibration,pore_pressure,temperature_c
0,45,5,0.030,100,29
1,20,5,0.010,60,27
2,55,30,0.140,170,34
3,30,10,0.030,90,29
4,15,2,0.005,50,26
5,40,0,0.010,80,28
6,60,35,0.160,180,35
7,25,8,0.020,70,28
8,50,20,0.100,150,32
9,35,12,0.050,120,30


In [ ]:
n_synthetic = 1000
np.random.seed(42)
synthetic_rows = []

for _ in range(n_synthetic):
    row = {}
    row["slope_angle"] = np.clip(np.random.choice(df_small["slope_angle"]) + np.random.randint(-10,11), 10, 70)
    row["rainfall_mm"] = np.clip(int(row["slope_angle"] * 0.5 + np.random.randint(-5,6)), 0, 50)
    row["vibration"] = np.clip(np.random.choice(df_small["vibration"]) + np.random.uniform(-0.02,0.02), 0, 0.2)
    row["pore_pressure"] = np.clip(int(row["rainfall_mm"]*5 + np.random.randint(-10,11)), 40, 200)
    row["temperature_c"] = np.clip(np.random.choice(df_small["temperature_c"]) + np.random.randint(-3,4), 25, 36)
    synthetic_rows.append(row)

df_synthetic = pd.DataFrame(synthetic_rows)
df_synthetic.tail()


,slope_angle,rainfall_mm,vibration,pore_pressure,temperature_c
995,55,26,0.140047,134,28
996,41,25,0.034673,121,36
997,19,6,0.110277,40,33
998,44,25,0.145675,129,32
999,24,10,0.111489,60,32


In [ ]:
# Numeric risk
df_synthetic["risk"] = 0.3*df_synthetic["slope_angle"] + 0.3*df_synthetic["rainfall_mm"] + 0.2*df_synthetic["vibration"]*100 + 0.2*df_synthetic["pore_pressure"]/10
df_synthetic["risk"] = df_synthetic["risk"]/df_synthetic["risk"].max()

# Low / Medium / High
def risk_level(x):
    if x < 0.3: return "Low"
    elif x < 0.6: return "Medium"
    else: return "High"

df_synthetic["risk_level"] = df_synthetic["risk"].apply(risk_level)
df_synthetic.head()


,slope_angle,rainfall_mm,vibration,pore_pressure,temperature_c,risk,risk_level
0,69,39,0.023946,191,36,0.939297,High
1,10,7,0.015715,40,27,0.159053,Low
2,31,14,0.022170,71,25,0.393221,Medium
3,40,17,0.180569,77,25,0.569517,Medium
4,70,38,0.140531,193,27,1.000000,High


In [ ]:
features = ["slope_angle","rainfall_mm","vibration","pore_pressure","temperature_c"]
X = df_synthetic[features]
y = df_synthetic["risk_level"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.945
              precision    recall  f1-score   support

        High       0.96      0.95      0.96        84
         Low       0.96      0.88      0.92        25
      Medium       0.93      0.96      0.94        91

    accuracy                           0.94       200
   macro avg       0.95      0.93      0.94       200
weighted avg       0.95      0.94      0.94       200



In [ ]:
import joblib
joblib.dump(model, "rockfall_model.pkl")


['rockfall_model.pkl']

In [ ]:
# Take first 10 samples from test set
results = X_test.copy()
results["Actual"] = y_test
results["Predicted"] = y_pred
results.head(10)


,slope_angle,rainfall_mm,vibration,pore_pressure,temperature_c,Actual,Predicted
521,56,23,0.145714,121,36,High,High
737,49,21,0.014457,108,32,High,Medium
740,57,30,0.168699,155,31,High,High
660,55,24,0.009174,123,29,High,High
411,28,16,0.159057,70,25,Medium,Medium
678,39,20,0.025183,98,28,Medium,Medium
626,38,23,0.085724,113,30,Medium,Medium
513,44,26,0.000791,135,36,High,High
859,43,23,0.004882,122,27,Medium,High
136,33,16,0.003031,75,36,Medium,Medium
